1️⃣ Instalación de Librerías
Antes de ejecutar el código, es necesario instalar las librerías requeridas. Se usa pip install para instalar pandas, numpy, scikit-learn y nltk, que se utilizan para manipulación de datos, cálculos matemáticos y procesamiento de texto.

In [1]:
# 📌 Instalación de librerías necesarias
!pip install pandas numpy scikit-learn nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2️⃣ Importación de Librerías
Se importan las librerías necesarias para el análisis y transformación de datos:

pandas: Para manipular DataFrames.
numpy: Para operaciones numéricas.
sklearn.feature_extraction.text: Para transformar texto en vectores numéricos (TF-IDF).
sklearn.metrics.pairwise: Para calcular similitudes entre vectores.
nltk: Para procesamiento de lenguaje natural.

In [2]:
# 📌 Importar Librerías Necesarias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

3️⃣ Carga de Datos
Los datos provienen de múltiples archivos CSV almacenados en la carpeta "CSV/Por Marcas". Se obtiene la lista de archivos CSV, se leen y combinan en un solo DataFrame (df_celulares) utilizando pandas.concat(). Esto permite tener una única tabla con toda la información.

In [3]:
# 📌 Cargar los Datos desde archivos CSV
import os
ruta_archivos = 'CSV/Por Marcas'  # Ruta donde están almacenados los archivos CSV

# Obtener la lista de archivos en la carpeta especificada
archivos = [archivo for archivo in os.listdir(ruta_archivos) if archivo.endswith('.csv')]

# Leer y unir todos los CSV en un solo DataFrame
df_lista = []
for archivo in archivos:
    df = pd.read_csv(os.path.join(ruta_archivos, archivo))  # Leer cada archivo CSV
    df_lista.append(df)

df_celulares = pd.concat(df_lista, ignore_index=True)  # Unir todos los DataFrames en uno solo

4️⃣ Eliminación de Columnas Innecesarias
Para simplificar el análisis, se eliminan las columnas de precios en diferentes países (Pakistan, India, China, Dubai) y se conserva solo el precio en EE.UU. (Launched Price (USA)).

In [4]:
# 📌 Eliminar Columnas de Precios No Deseadas
columnas_a_eliminar = ['Launched Price (Pakistan)', 'Launched Price (India)', 'Launched Price (China)', 'Launched Price (Dubai)']
df_celulares.drop(columns=[col for col in columnas_a_eliminar if col in df_celulares.columns], inplace=True)  # Eliminar solo si existen

5️⃣ Renombrado de Columnas
Se renombran las columnas con nombres más claros y estandarizados, como "Mobile Weight (g)", "RAM (GB)", "Battery Capacity (mAh)", etc., facilitando su manipulación.

In [5]:
# 📌 Renombrar Columnas para un mejor entendimiento
df_celulares.columns = ['Company Name', 'Model Name', 'Mobile Weight (g)', 'RAM (GB)', 'Front Camera (MP)',
                        'Back Camera (MP)', 'Processor', 'Battery Capacity (mAh)', 'Screen Size (inches)', 'Launched Price (USA)', 'Launched Year']

6️⃣ Limpieza de Datos
Se eliminan las filas con valores nulos (dropna()).
Se convierten a minúsculas los valores de "Model Name" y "Processor" para estandarizar.
Se eliminan las letras innecesarias en "Mobile Weight (g)" y se convierte a float.

In [6]:
# 📌 Limpieza y Conversión de Datos

df_celulares.dropna(inplace=True)  # Eliminar filas con valores nulos

# Convertir nombres de modelos y procesadores a minúsculas para estandarizar
df_celulares['Model Name'] = df_celulares['Model Name'].str.lower()
df_celulares['Processor'] = df_celulares['Processor'].str.lower()

# Convertir pesos de móviles a valores numéricos
df_celulares['Mobile Weight (g)'] = df_celulares['Mobile Weight (g)'].str.replace('g', '', regex=True).astype(float)

7️⃣ Expansión de Variantes de RAM
Algunos celulares tienen múltiples opciones de RAM (4GB/6GB/8GB). Se separan estos valores y se crean filas individuales para cada variante de RAM.

In [7]:
# 📌 Expandir Variantes de RAM en diferentes filas

def expandir_variantes_ram(df):
    """
    Función que separa valores de RAM expresados en múltiplas variantes (ej. "4GB/6GB/8GB")
    y los convierte en filas individuales.
    """
    nuevas_filas = []
    for _, fila in df.iterrows():
        valores_ram = [float(x) for x in str(fila['RAM (GB)']).replace('GB', '').split('/') if x.strip().isdigit()]
        for ram in valores_ram:
            nueva_fila = fila.copy()
            nueva_fila['RAM (GB)'] = ram
            nuevas_filas.append(nueva_fila)
    return pd.DataFrame(nuevas_filas)

df_celulares = expandir_variantes_ram(df_celulares)

8️⃣ Extracción de Megapíxeles de las Cámaras
Las columnas "Front Camera (MP)" y "Back Camera (MP)" pueden contener múltiples valores (12MP/16MP). Se extrae el primer valor encontrado y se convierte en float.

In [8]:
# 📌 Extraer megapíxeles de las cámaras

def extraer_megapixeles(valor):
    """
    Función que extrae el valor numérico de megapíxeles de una cadena de texto (ej. "12MP/16MP").
    Devuelve el primer valor encontrado.
    """
    valores = [float(x) for x in valor.replace('MP', '').split('/') if x.strip().replace('.', '').isdigit()]
    return valores[0] if valores else np.nan

df_celulares['Front Camera (MP)'] = df_celulares['Front Camera (MP)'].astype(str).apply(extraer_megapixeles)
df_celulares['Back Camera (MP)'] = df_celulares['Back Camera (MP)'].astype(str).apply(extraer_megapixeles)

9️⃣ Conversión de Capacidad de Batería
La columna "Battery Capacity (mAh)" puede contener comas (4,500mAh). Se eliminan y se convierte el valor en número flotante.

In [9]:
# 📌 Limpieza y conversión de la capacidad de batería
df_celulares['Battery Capacity (mAh)'] = (
    df_celulares['Battery Capacity (mAh)']
    .str.replace(',', '', regex=True)  # Eliminar comas para valores numéricos en miles
    .str.replace('mAh', '', regex=True)  # Eliminar la unidad de medida
    .astype(float)  # Convertir a número flotante
)

🔟 Extracción del Tamaño de Pantalla
Algunos valores en "Screen Size (inches)" incluyen anotaciones adicionales (6.7 (unfolded)). Se extrae solo el número.

In [10]:
# 📌 Extraer el tamaño de pantalla
def extraer_tamano_pantalla(valor):
    """
    Función que extrae el tamaño numérico de la pantalla desde una cadena de texto,
    ignorando posibles anotaciones adicionales como "(unfolded)".
    """
    import re
    match = re.search(r'\d+(\.\d+)?', valor)
    return float(match.group()) if match else np.nan

df_celulares['Screen Size (inches)'] = df_celulares['Screen Size (inches)'].astype(str).apply(extraer_tamano_pantalla)

1️⃣1️⃣ Limpieza y Conversión del Precio
Se eliminan comas (2,000 → 2000).
Se elimina el prefijo "USD " para dejar solo los valores numéricos.
Se convierte el precio a float.

In [11]:
# 📌 Limpieza y conversión del precio de lanzamiento
df_celulares['Launched Price (USA)'] = (
    df_celulares['Launched Price (USA)']
    .str.replace(',', '', regex=True)  # Eliminar comas para valores en miles
    .str.replace('USD ', '', regex=True)  # Eliminar prefijo "USD"
    .astype(float)  # Convertir a número flotante
)

1️⃣2️⃣ Visualización de Datos Limpios
Se imprime un resumen de los datos procesados utilizando display(df_celulares.head()).

In [12]:
# 📌 Mostrar los datos después de la limpieza
print("Datos después de la limpieza y conversión:")
display(df_celulares.head())

Datos después de la limpieza y conversión:


,Company Name,Model Name,Mobile Weight (g),RAM (GB),Front Camera (MP),Back Camera (MP),Processor,Battery Capacity (mAh),Screen Size (inches),Launched Price (USA),Launched Year
0,Apple,iphone 16 128gb,174.0,6.0,12.0,48.0,a17 bionic,3600.0,6.1,799.0,2024
1,Apple,iphone 16 256gb,174.0,6.0,12.0,48.0,a17 bionic,3600.0,6.1,849.0,2024
2,Apple,iphone 16 512gb,174.0,6.0,12.0,48.0,a17 bionic,3600.0,6.1,899.0,2024
3,Apple,iphone 16 plus 128gb,203.0,6.0,12.0,48.0,a17 bionic,4200.0,6.7,899.0,2024
4,Apple,iphone 16 plus 256gb,203.0,6.0,12.0,48.0,a17 bionic,4200.0,6.7,949.0,2024
